In [3]:
# triage.py
from abc import ABC, abstractmethod

class Paciente(ABC):
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad
    
    @abstractmethod
    def resumen(self):
        pass

    @abstractmethod
    def prioridad(self):
        pass

    def calcular_imc(self, peso, altura):
        if altura > 0:
            return peso / (altura ** 2)
        return 0

class PacienteElectivo(Paciente):
    def __init__(self, nombre, edad, motivo):
        super().__init__(nombre, edad)
        self.motivo = motivo
    
    def resumen(self):
        return f"Paciente Electivo: {self.nombre}, {self.edad} años. Motivo: {self.motivo}"
    
    def prioridad(self):
        return 1

class PacienteUrgente(Paciente):
    def __init__(self, nombre, edad, motivo, saturacion_o2):
        super().__init__(nombre, edad)
        self.motivo = motivo
        self.saturacion_o2 = saturacion_o2
    
    def resumen(self):
        return f"Paciente Urgente: {self.nombre}, {self.edad} años. Motivo: {self.motivo}. SatO2: {self.saturacion_o2}%"
    
    def prioridad(self):
        return 100 - self.saturacion_o2

class TrasladoAmbulancia(Paciente):
    mapa_severidad = {
        'rojo': 100,
        'amarillo': 70,
        'verde': 40
    }
    
    def __init__(self, folio, nombre, codigo_trauma):
        super().__init__(nombre, 0)
        self.folio = folio
        self.codigo_trauma = codigo_trauma.lower()
    
    def resumen(self):
        return f"Traslado en Ambulancia (Folio {self.folio}): {self.nombre}. Código: {self.codigo_trauma.upper()}"
    
    def prioridad(self):
        return self.mapa_severidad.get(self.codigo_trauma, 0)

class PacienteTriage(Paciente):
    def __init__(self, nombre, edad, peso, altura):
        super().__init__(nombre, edad)
        self.peso = peso
        self.altura = altura
    
    def resumen(self):
        imc = self.calcular_imc(self.peso, self.altura)
        return f"Paciente Triage: {self.nombre}, {self.edad} años. IMC: {imc:.2f}"
    
    def prioridad(self):
        imc = self.calcular_imc(self.peso, self.altura)
        prioridad_base = 50
        
        if imc < 18.5:
            return prioridad_base + 30
        elif imc >= 30:
            return prioridad_base + 40
        return prioridad_base

In [4]:
class Hospital:
    def __init__(self, nombre, camas):
        self.nombre = nombre
        self.camas_totales = camas
        self.camas_ocupadas = 0
        self.admitidos = []
        self.rechazados = []
    
    def camas_disponibles(self):
        return self.camas_totales - self.camas_ocupadas
    
    def intentar_admitir(self, ingreso):
        if self.camas_disponibles() > 0:
            self.admitidos.append(ingreso)
            self.camas_ocupadas += 1
            print(f"Admitido: {ingreso.resumen()}")
        else:
            self.rechazados.append(ingreso)
            print(f"Rechazado: {ingreso.resumen()}")

In [5]:
hospital = Hospital("Hospital Central", camas=3)

ingresos_mixtos = [
    PacienteElectivo("Juan Pérez", 65, "Chequeo de rutina"),
    PacienteUrgente("María López", 42, "Dolor de pecho", 90),
    TrasladoAmbulancia("A101", "Carlos Gómez", "rojo"),
    PacienteTriage("Laura Torres", 28, 95, 1.65),
    PacienteUrgente("Pedro García", 78, "Dificultad respiratoria", 75),
    PacienteElectivo("Ana Silva", 33, "Cirugía programada"),
    PacienteTriage("Felipe Rojas", 55, 60, 1.80),
    TrasladoAmbulancia("A102", "Sofía Mendoza", "amarillo")
]

ingresos_mixtos.sort(key=lambda x: x.prioridad(), reverse=True)

print("--- Proceso de Triage ---")
for ingreso in ingresos_mixtos:
    hospital.intentar_admitir(ingreso)

--- Proceso de Triage ---
Admitido: Traslado en Ambulancia (Folio A101): Carlos Gómez. Código: ROJO
Admitido: Paciente Triage: Laura Torres, 28 años. IMC: 34.89
Admitido: Traslado en Ambulancia (Folio A102): Sofía Mendoza. Código: AMARILLO
Rechazado: Paciente Triage: Felipe Rojas, 55 años. IMC: 18.52
Rechazado: Paciente Urgente: Pedro García, 78 años. Motivo: Dificultad respiratoria. SatO2: 75%
Rechazado: Paciente Urgente: María López, 42 años. Motivo: Dolor de pecho. SatO2: 90%
Rechazado: Paciente Electivo: Juan Pérez, 65 años. Motivo: Chequeo de rutina
Rechazado: Paciente Electivo: Ana Silva, 33 años. Motivo: Cirugía programada


In [6]:
print("\n--- Reporte Final ---")
print(f"Camas totales: {hospital.camas_totales}")
print(f"Camas ocupadas: {hospital.camas_ocupadas}")
print(f"Camas disponibles: {hospital.camas_disponibles()}")


--- Reporte Final ---
Camas totales: 3
Camas ocupadas: 3
Camas disponibles: 0


In [7]:
nombres_admitidos = [p.nombre for p in hospital.admitidos]
nombres_rechazados = [p.nombre for p in hospital.rechazados]

print("Admitidos:")
print(nombres_admitidos)
print("\nRechazados:")
print(nombres_rechazados)

Admitidos:
['Carlos Gómez', 'Laura Torres', 'Sofía Mendoza']

Rechazados:
['Felipe Rojas', 'Pedro García', 'María López', 'Juan Pérez', 'Ana Silva']
